 # **TEMAT PROJEKTU:**   Histopathologic Cancer Detection

**AUTORZY:**  
Paulina Radomska  
Aleksandra Wirecka  
Marta Denisiuk  

# **CEL PROJEKTU:**
Stworzenie modelu, który może analizować skany węzłów chłonnych i przewidywać, czy obraz zawiera tkankę przerzutową, czyli raka.

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import cv2
import os

from sklearn.utils import shuffle   # tablice losowania/wykonywania losowych permutacji kolekcji
from sklearn.model_selection import train_test_split    # dzielenie tablic lub macierzy na losowe podzbiory pociągów i testów
import shutil   # operacje na plikach
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix   # oblicza obszar pod krzywą charakterystyki

import plotly.graph_objects as go

import plotly.figure_factory as ff
%matplotlib inline

tf.random.set_seed(101)

# **DANE I ICH WSTĘPNA ANALIZA**

In [ ]:
path = "../input/histopathologic-cancer-detection/"

# ładujemy zestawy szkoleniowe
train = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
# ile obrazków ma każdy z zestawów?
print(len(train))
print(len(test))

In [ ]:
train.head()

In [ ]:
df_data=train

In [ ]:
directory = path + "train/"
fnames = [os.path.join(directory, fname) for fname in os.listdir(directory)]
slownik = {'png' : 0, 'jpg': 0, 'jpeg' : 0, 'tiff': 0, 'bmp' :0,'tif':0}

for i in fnames:
    if i.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'tif')):
        slownik[(i.split('.')[3])] += 1
print(slownik)

#widzimy, że mamy same tif, więc sprawdzamy czy wszystkie zdjęcia zostały w tym formacie zapisane,
if len(fnames) == slownik['tif']:
    print('True')
# czyli wszystkie są w tif

In [ ]:
# tworzymy ramkę danych obrazów szkoleniowych
print(df_data.shape)
df_data['id']=df_data['id'].apply(lambda x: x+'.tif')
df_data.head()

Zbiór danych zawiera 220 025 obrazów do treningu, które są oznaczone 0 lub 1.  
0 oznacza wynik negatywny, tj. brak raka, a 1 oznacza wynik pozytywny, tj. obraz zawiera przerzuty (raka).  

Zbiór danych zawiera również 57 458 obrazów testowych, które posłużą do oceny przesłanych przez nas materiałów. Próbki testowe są nieoznaczone, więc nie będą używane do budowy i oceny naszego modelu.

In [ ]:
count = df_data['label'].value_counts()

print("Ilość pozytywnych skanów raka:", count[1])
print("Ilość pozytywnych skanów raka w procentach:", round(count[1] / df_data.shape[0], 3) * 100)
print("Ilość neatywnych skanów raka:", count[0])
print("Ilość negatywnych skanów raka w procentach:", round(count[0] / df_data.shape[0], 3) * 100)

Zbiór danych szkoleniowych składa się ze 130 908 negatywów i 89 117 pozytywów, co stanowi odpowiednio około 59.5% i 40.5%, jak pokazano na poniższym wykresie kołowym.  

In [ ]:
labels = ['Negatywy', 'Pozytywy']
fig1, ax1 = plt.subplots()
colors = ['#66b3ff','#ff6666']
ax1.pie(count, labels=labels,autopct='%1.1f%%',startangle=90,colors=colors)  
plt.show()

# **WIZUALIZACJA OBRAZÓW**

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(8, 5), dpi=150)


images = []
for i in range(3):
    for j in range(3):
        
        tran = np.random.randint(0,1000)
                
        image = cv2.imread(path + "train/" + df_data.iloc[tran]['id'])
        images.append(axs[i, j].imshow(image))
        
        if df_data.iloc[tran]['label'] == 1:
            axs[i,j].set_title('Nowotwór')
        else:
            axs[i,j].set_title('Brak nowotworu')
            
        axs[i,j].set_xticks([])
        axs[i,j].set_yticks([])
        

    
plt.show()
del images

Z przedstawionych obrazków trudno jest ustalić cechy odróżniające komórki nowotworowe od nienowotworowych.
Możemy zauważyć, że istnieją obrazy komórek rakowych i nierakowych o podobnych kolorach oraz z dużą i małą liczbą okrągłych węzłów.
Spójrzmy, jak kreśli się częstotliwość kanałów kolorów losowo wybranego obrazka dla dwóch możliwych kategorii.

In [ ]:
cancer_data = df_data[(df_data.label==1)]
cancer_image = cancer_data.iloc[900]['id']
img = cv2.imread(path + "train/" + cancer_image)
plt.imshow(img)
plt.title("Komórka rakowa")
plt.show()

In [ ]:
plt.hist(img[:, :, 0].ravel(), bins = 256, color = 'red')
plt.hist(img[:, :, 1].ravel(), bins = 256, color = 'Green')
plt.hist(img[:, :, 2].ravel(), bins = 256, color = 'Blue')
plt.xlabel('Intensywność')
plt.ylabel('Ilość')
plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.title("Częstotliwość kanałów kolorów komórek rakowych")
plt.show()


In [ ]:
non_cancer_data = df_data[(df_data.label==0)]
non_cancer_image = non_cancer_data.iloc[500]['id']

img = cv2.imread(path + "train/" + non_cancer_image)
plt.imshow(img)
plt.title("Brak komórki rakowej")
plt.show()

In [ ]:
plt.hist(img[:, :, 0].ravel(), bins = 256, color = 'red')
plt.hist(img[:, :, 1].ravel(), bins = 256, color = 'Green')
plt.hist(img[:, :, 2].ravel(), bins = 256, color = 'Blue')
plt.xlabel('Intensywność')
plt.ylabel('Ilość')
plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.title("Częstotliwość kanałów kolorów przy braku komórek rakowych")
plt.show()

del img, non_cancer_data, cancer_data

# **PODZIAŁ NA ZBIÓR TRENINGOWY I WALIDACYJNY**

Ponieważ zbiór danych jest bardzo duży, a uczenie modelu przy użyciu całego zestawu danych byłoby bardzo czasochłonne, zdecydowałyśmy się zmniejszyć liczbę próbek uczących, które będą używane.  
Opierając się na wykonanych próbach i pouczających błędach zdecydowałyśmy się na użycie łącznie 20 000 próbek, tj.  
10 000 pozytywnych i 10 000 negatywnych.  
Zadany rozmiar próbki powinien być wystarczająco duży, aby reprezentować pełny zestaw danych uczących, jednocześnie umożliwiając stosunkowo szybkie wytrenowanie modelu. 

Ustaliłyśmy, aby zrównać liczbę pozytywnych i negatywnych próbek, by dokładność nie została wypaczona przez proste dopasowanie modelu do częstszego wyniku.

In [ ]:
# zbiór danych zawiera obrazy histopatologiczne, każdy obraz ma rozmiar 96px * 96px
# ustawiamy wymagania wstępne
sample_size = 10000     # ilość próbek do szkolenia
image_size = 96         # rozmiar obrazka
image_channels = 3      # kanały obrazu

In [ ]:
df_neg = df_data[df_data['label'] == 0].sample(sample_size, random_state = 101)
df_pos = df_data[df_data['label'] == 1].sample(sample_size, random_state = 101)

data = pd.concat([df_neg, df_pos], axis=0).reset_index(drop=True)   # złączenie obiektów negatywnych i pozytywnych w jedną ramkę danych
data = shuffle(data)    # przetasowanie listy (zreorganizowanie kolejności elementów listy)

print(data['label'].value_counts())
print(data.shape)
print(data)

In [ ]:
test['label'].value_counts()

Ponieważ obrazy testowe (zmienna *test*) nie miały etykiet innych niż *label=0*, podzieliłyśmy nasze 20000 próbek szkoleniowych na zestaw treningowy i walidacyjny.  
Zdecydowałyśmy się wykorzystać 90% obrazów do trenowania modelu, a 10% zarezerwować do jego walidacji.  

Próbki podzielono losowo. Zdecydowałyśmy się na metodę stratyfikację, aby jeszcze raz zachować liczbę pozytywów i negatywów w każdym zestawie, aby model nie był wypaczony w kierunku najczęstszego wyniku.

In [ ]:
data_train, data_valid = train_test_split(data, test_size=.1, random_state=101, stratify=data['label'])
# stratify tworzy zrównoważony zestaw walidacyjny

print(data_train.shape)
print(data_valid.shape)

In [ ]:
print(data_train['label'].value_counts())
print(data_valid['label'].value_counts())

Ustaliłyśmy, że łatwiejsze niż prowadzenie listy obrazków i ciągłe jej czytanie, będzie stworzenie katalogów/folderów.  
Utworzyłyśmy katalogi do przechowywania obrazów, odpowiednio katalog treningowy i walidacyjny. Następnie obrazy zostały posortowane do odpowiednich folderów.

In [ ]:
# nowy katalog
base_dir='base_dir'
os.mkdir(base_dir)

In [ ]:
# tworzymy dwa foldery wewnątrz "base_dir"
# train_dir
    # a_non_cancer_tissue
    # b_cancer_tissue

# valid_dir
    # a_non_cancer_tissue
    # b_cancer_tissue
    
# ścieżka do "base_dir", by dołączyć nazwy nowych folderów
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# valid_dir
valid_dir = os.path.join(base_dir, 'valid_dir')
os.mkdir(valid_dir)

In [ ]:
# w każdym folderze tworzymy osobne foldery dla każdej klasy - a_non_cancer_tissue, b_cancer_tissue
# dla train_dir
non_cancer_tissue = os.path.join(train_dir, 'a_non_cancer_tissue')
os.mkdir(non_cancer_tissue)

cancer_tissue = os.path.join(train_dir, 'b_cancer_tissue')
os.mkdir(cancer_tissue)


# dla valid_dir
non_cancer_tissue = os.path.join(valid_dir, 'a_non_cancer_tissue')
os.mkdir(non_cancer_tissue)
cancer_tissue = os.path.join(valid_dir, 'b_cancer_tissue')
os.mkdir(cancer_tissue)

In [ ]:
# sprawdzamy, czy foldery rzeczywiście istnieją
os.listdir('base_dir/valid_dir')

In [ ]:
# ustawiamy id jako indeks w zmiennej data
data.set_index('id', inplace=True)

In [ ]:
# lista obrazów treningowych i walidacyjnych
train_list = list(data_train['id'])
valid_list = list(data_valid['id'])


In [ ]:
# przenosimy obrazy do folderów
for image in train_list:
    
    # wydobycie etykiety (label)
    tlab = data.loc[image,'label']
    
    if tlab == 0:
        label = 'a_non_cancer_tissue'
    if tlab == 1:
        label = 'b_cancer_tissue'
    
    # ścieżka źródłowa
    src = os.path.join('../input/histopathologic-cancer-detection/train', image)
    # docelowa ścieżka
    dst = os.path.join(train_dir, label, image)
    # skopiowanie obrazu ze źródła do miejsca docelowego
    shutil.copyfile(src, dst)

In [ ]:
print(len(os.listdir('base_dir/train_dir/a_non_cancer_tissue')))
print(len(os.listdir('base_dir/train_dir/b_cancer_tissue')))

In [ ]:
for image in valid_list:
    
    tlab = data.loc[image,'label']
    
    if tlab == 0:
        label = 'a_non_cancer_tissue'
    if tlab == 1:
        label = 'b_cancer_tissue'
    
    src = os.path.join('../input/histopathologic-cancer-detection/train', image)
    dst = os.path.join(valid_dir, label, image)
    shutil.copyfile(src, dst)

In [ ]:
print(len(os.listdir('base_dir/valid_dir/a_non_cancer_tissue')))
print(len(os.listdir('base_dir/valid_dir/b_cancer_tissue')))

Obrazy zostały wstępnie przetworzone przy użyciu funkcji *keras ImageDataGenerator* ("powiększanie/rozszerzenie danych w czasie rzeczywistym") 

Metoda *flow_from_directory()* umożliwia odczytywanie obrazów bezpośrednio z katalogu i powiększanie ich podczas uczenia się modelu sieci neuronowej na danych szkoleniowych.  
Metoda oczekuje, że obrazy należące do różnych klas są obecne w różnych folderach, ale znajdują się wewnątrz tego samego folderu nadrzędnego.

In [ ]:
train_path = path + 'train/'
test_path = path + 'test/'

num_train_samples = len(data_train)
num_valid_samples = len(data_valid)
train_batch_size = 32
valid_batch_size = 32

train_steps = np.ceil(num_train_samples / train_batch_size)
valid_steps = np.ceil(num_valid_samples / valid_batch_size)
# ceil zwraca tzw. "sufit", górne zaokrąglenie

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True, horizontal_flip=True, rotation_range=90, shear_range=0.05)

# rescale - oryginalne obrazy składają się ze współczynników RGB w 0-255, ale takie wartości są zbyt wysokie 
# dla naszego modelu do przetwarzania, więc oczekujemy wartości od 0 do 1 (skalując za pomocą 1/255)
# vertical_flip - losowe przerzucanie danych wejściowych w pionie
# horizontal_flip - losowe przerzucanie danych wejściowych w poziomie
# rotation_range - zakres stopni dla losowych obrotów
# shear_range - intensywność ścinania (kąt ścinania w kierunku przeciwnym do ruchu wskazówek zegara w stopniach)

# katalog - ścieżka do folderu nadrzędnego, który zawiera podfolder dla różnych obrazów klas
# target_size - rozmiar obrazu wejściowego
# batch_size - rozmiar partii danych
# class_mode - dla etykiet binarnych 'binary'/dla etykiet zakodowanych 'categorical'

train_data_gen = datagen.flow_from_directory(train_dir,
                                        target_size=(image_size,image_size),
                                        batch_size=train_batch_size,
                                        class_mode='binary')

valid_data_gen = datagen.flow_from_directory(valid_dir,
                                        target_size=(image_size,image_size),
                                        batch_size=valid_batch_size,
                                        class_mode='binary')


test_data_gen = datagen.flow_from_directory(valid_dir,
                                        target_size=(image_size,image_size),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=False)

# **MODEL**

In [ ]:
kernel_size = (3,3) # wysokość i szerokość okna splotu
filters = 32 # wymiar przestrzeni wyjściowej

model = Sequential()
model.add(Conv2D(filters, kernel_size, activation = 'relu', input_shape = (image_size, image_size, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size, use_bias=False))
model.add(MaxPooling2D(pool_size=(2, 2)))
# normalizacja danych wejściowych
model.add(BatchNormalization())
model.add(Activation("relu")) 
model.add(Dropout(0.3))
# Dropout jest techniką, w której losowo wybrane neurony są ignorowane podczas treningu. 
# Ich wkład w aktywację neuronów niższego rzędu jest czasowo usuwany na przejściu do przodu i wszelkie aktualizacje wagi nie są stosowane.

# spłaszczenie danych
model.add(Flatten())
# zwykła gęsto połączona warstwa
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model

# wykres warstw modelu
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# zmniejsza współczynnik uczenia się, gdy wskaźnik przestaje się poprawiać
reduce = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5, mode='min')

mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
%%time

history = model.fit(train_data_gen, steps_per_epoch=train_steps, 
                    validation_data=valid_data_gen,
                    validation_steps=valid_steps,
                    epochs=10,
                   callbacks=[mc, reduce])

Wygenerowałyśmy prognozy dla próbek walidacyjnych i porównałyśmy te prognozy z rzeczywistymi wynikami.  
Poniższa krzywa ROC przedstawia wskaźniki prawdziwie pozytywne i fałszywie dodatnie. 

Generalnie celem problemów klasyfikacyjnych jest maksymalizacja pola powierzchni pod krzywą ROC, próbując uzyskać wartość możliwie jak najbliższą 1.

In [ ]:
pred = model.predict(test_data_gen, steps=len(test_data_gen), verbose=1)
fpr, tpr, thresholds_keras = roc_curve(test_data_gen.classes, pred)
roc_auc_score(test_data_gen.classes, pred)

In [ ]:
pred

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1])
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Accuracy vs. Val Accuracy', xaxis_title='Epoki', yaxis_title='Accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Loss vs. Val Loss', xaxis_title='Epoki', yaxis_title='Loss', yaxis_type='log')
    fig.show()

plot_hist(history)

In [ ]:
y_pred=[]
y_true=[]
def predict_img(model, img):
    X=np.array(img)/255
    X=cv2.resize(X,(image_size,image_size))
    X=np.expand_dims(X,axis=0)
    pred_class=model.predict_classes(X)
    #print(pred_class)
    y_pred.append(pred_class[0][0])
    y_true.append(temp['label'])
    if pred_class == 0:
        plt.title("Non cancer\n" + str(temp['label']))
    else:
        plt.title("Cancer\n" + str(temp['label']))
    plt.imshow(img)

In [ ]:
plt.figure(figsize=(13, 13))
for i in range(1,11):
    temp = df_data.iloc[np.random.randint(0,100)]
    img = cv2.imread(path + "train/" + temp['id'])
    plt.subplot(1, 10, i)
    plt.axis('off')
    predict_img(model,img)
plt.show()

In [ ]:
y_pred=[]
y_true=[]
for i in range(1,101):
    temp = df_data.iloc[np.random.randint(350,450)]
    img = cv2.imread(path + "train/" + temp['id'])
    predict_img(model,img)
    

In [ ]:
cm=confusion_matrix(y_true,y_pred)
cm

In [ ]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='Hot', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)